In [1]:
import requests
from bs4 import BeautifulSoup

# Headers for the HTTP request
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.airlinequality.com/airline-reviews/british-airways/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

# URL and page settings
url = "https://www.airlinequality.com/airline-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=4000"
# url2 = "https://www.airlinequality.com/seat-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=4000"
# url3 = "https://www.airlinequality.com/lounge-reviews/british-airways/page/{}/?sortby=post_date%3ADesc&pagesize=4000"

page = 1

# Fetch the webpage content
response = requests.get(url.format(page), headers=headers)
# response2 = requests.get(url2.format(page), headers=headers)
# response3 = requests.get(url3.format(page), headers=headers)

soup = BeautifulSoup(response.text, "html.parser")
# soup2 = BeautifulSoup(response2.text, "html.parser")
# soup3 = BeautifulSoup(response3.text, "html.parser")

airline_reviews = soup.find_all("article", attrs={"itemprop": "review"})
# seat_reviews = soup2.find_all("article", attrs={"itemprop": "review"})
# lounge_reviews = soup3.find_all("article", attrs={"itemprop": "review"})


In [62]:
# Helper function to extract star ratings
def extract_ratings(review, class_name):
    ratings = []
    table = review.table
    if table:
        td_elements = table.find_all("td", class_=class_name)
        if td_elements:
            for td in td_elements:
                next_sibling = td.find_next_sibling()
                if next_sibling:
                    filled_stars = next_sibling.find_all('span', class_='star fill')
                    if filled_stars:
                        last_filled_star = filled_stars[-1].get_text(strip=True)
                        ratings.append(int(last_filled_star))
                    else:
                        ratings.append('NaN')
                else:
                    ratings.append('NaN')
        else:
            ratings.append('NaN')
    else:
        ratings.append('NaN')
    return ratings

In [63]:
# Define the review categories
categories = [
    "seat_comfort",
    "ground_service",
    "value_for_money",
    "cabin_staff_service",
    "food_and_beverages",
    "inflight_entertainment",
    "wifi_and_connectivity",
    "recommended"
]

airline_ratings_dict = {category: [] for category in categories}

In [64]:
# Extract ratings for each category
for review in airline_reviews:
    for category in categories:
        ratings = extract_ratings(review, category)
        airline_ratings_dict[category].extend(ratings)

In [65]:
# Process 'recommended' category separately as it does not follow the star rating pattern
recommended = []
for review in airline_reviews:
    table = review.table
    if table:
        td_elements = table.find_all("td", class_="recommended")
        for td in td_elements:
            next_sibling = td.find_next_sibling()
            if next_sibling:
                review_value = next_sibling.get_text(strip=True).lower()
                if review_value == "no":
                    recommended.append(0)
                else:
                    recommended.append(1)
    else:
        recommended.append('NaN')
airline_ratings_dict["recommended"] = recommended

In [66]:
aircraft=[]
for review in airline_reviews:
    table = review.table
    if table:
        td_elements = table.find_all("td", class_="aircraft")
        # print(td_elements)
        if td_elements:
            for td in td_elements:
                next_sibling = td.find_next_sibling()
                # print(next_sibling)
                review_value = next_sibling.get_text(strip=True).lower().replace(" ",'')
                aircraft.append(review_value)
        else:
            aircraft.append('NaN')
#                 if next_sibling:
#                     review_value = next_sibling.get_text(strip=True).lower().replace(" ",'')
#                     aircraft.append(review_value)
#                 else:
#                     aircraft.append("None")
#         else:
#             aircraft.append("None")
#     else:
#         aircraft.append("None")
airline_ratings_dict["aircraft"]=aircraft

In [99]:
reviewBody=[]
for review in airline_reviews:
    r=review.find_all("div",attrs={'itemprop':'reviewBody'})
    for i in r:
        reviewBody.append(i.text.replace("✅ Trip Verified","").replace("Not Verified","").replace("|" ,""))
# len(reviewBody)
airline_ratings_dict["reviewBody"]=reviewBody

In [93]:
reviewHead=[]
for review in airline_reviews:
    r=review.find_all("div",attrs={'class':'body'})
    for i in r:
        reviewHead.append(i.h2.text)
# len(reviewHead)
airline_ratings_dict["reviewHead"]=reviewHead

In [100]:
# Print the collected ratings
for category, ratings in airline_ratings_dict.items():
    print(f'{category}: {len(ratings)}')

seat_comfort: 3808
ground_service: 3808
value_for_money: 3808
cabin_staff_service: 3808
food_and_beverages: 3808
inflight_entertainment: 3808
wifi_and_connectivity: 3808
recommended: 3808
aircraft: 3808
reviewHead: 3808
reviewBody: 3808


In [101]:
import pandas as pd

df=pd.DataFrame(airline_ratings_dict)

df.to_csv("./data/airline_reviews.csv",index=0)

all ratings are out of 5
<br>
recommended ratings are 0 for not recommended and 1 for recommended

In [102]:
df.head()

,seat_comfort,ground_service,value_for_money,cabin_staff_service,food_and_beverages,inflight_entertainment,wifi_and_connectivity,recommended,aircraft,reviewHead,reviewBody
0,1,1,2,NaN,NaN,NaN,NaN,0,NaN,"""You expect better from BA""",BA cancelled the flight from Tokyo to LHR. ...
1,1,1,1,1,NaN,NaN,NaN,0,boeing747,"“never fly British Airways""",I strongly advise everyone to never fly Brit...
2,4,1,2,3,2,4,NaN,0,NaN,“we will rethink BA moving forward”,My partner and I were on the BA2166 return f...
3,1,1,1,1,1,1,1,0,NaN,“extremely poor customer service”,We had a Premium Economy return flight Los ...
4,5,5,3,4,1,NaN,NaN,1,a320,“a pleasant and civilized journey”,I should like to review my flight with Brit...


3808

3808

In [104]:
reviewBody[0]

'   BA cancelled the flight from Tokyo to LHR. I was booked on next day flight. There was another flight on the same day. I went to the desk, but the flight was full. BA in charge offers another flight through Hong Kong which would have been 26h flight time. I declined that, and asked to stay on the next day flight. To my dismay he cancelled the next day flight without telling me he did that. I think he was annoyed that I didn’t accept the offer after he spent sometime looking for. In fact I am the one who should be annoyed for cancelling my flight. I ended up flying another airline with downgrading. Poor service, and appalling behaviour. You expect better from BA.'